In [4]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pickle

In [5]:
df = pd.read_csv('train.csv')
df = df[['Sex', 'Age', 'Fare', 'Pclass', 'Survived']]
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Age'] = df['Age'].fillna(df['Age'].median())

In [6]:
train, test = train_test_split(df, test_size=0.2)

In [7]:
def model_tree(max_depth, min_samples_split):
    # Создание и обучение модели решающего дерева
    model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split)
    model.fit(train.drop('Survived', axis=1), train['Survived'])

    # Вычисление метрики
    preds = model.predict(test.drop('Survived', axis=1))
    acc = accuracy_score(test['Survived'], preds)
    print('accuracy', acc)

    # Сохранение модели в формате pickle
    with open('model.pkl', 'wb') as f:
        pickle.dump(model, f)

In [8]:
# Гиперпараметры
max_depth = 5
min_samples_split = 150

# Вызов функции с гиперпараметрами
model_tree(max_depth, min_samples_split)

accuracy 0.8212290502793296


## Инференс

In [9]:
import pickle

# Загрузка модели из файла pickle
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [10]:
import pandas as pd

new_data = pd.DataFrame({
    'Sex': [0],
    'Age': [5],
    'Fare': [10],
    'Pclass': [3]
})

In [11]:
# Предсказание нового df
predictions = model.predict(new_data)

# Ответ. 0 не выжил, 1 выжил
print('Predicted survival', predictions)

Predicted survival [1]


## Test API

In [15]:
import requests

def predict_model(data):
    url = 'http://127.0.0.1:5000/predict_model'

    # Отправка POST-запросса с данными в формате форм-данных
    response = requests.post(url, json=data)

    # Проверка статуса ответа
    if response.status_code == 200:
        return response.json()
    else:
        return {"error":f"Request failed with status code{response.status_code}"}
    
# Пример данных для предсказания
data = {
    "Sex": 0,
    "Age": 35.0,
    "Fare": 10.0,
    "Pclass": 3
}

# Получение предсказания
prediction = predict_model(data)
print(prediction)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /predict_model (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001A829EC5C10>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))